<a href="https://colab.research.google.com/github/ManasiBorhade/Multiple-Disease-Prediction/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]  

train_path = '/content/drive/MyDrive/DLDataset/images'
test_path = '/content/drive/MyDrive/DLDataset/facemasktest'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
  # useful for getting number of classes
folders = glob('/content/drive/MyDrive/DLDataset/images/*')

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)



# fit the model
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

model.save('hero_model.model')

#prediction
import cv2

img = cv2.imread('/content/drive/MyDrive/DLDataset/images/HeroineImages/download1.jpg')
img = cv2.resize(img, (224, 224)).astype(np.float32)
#img = img.transpose((2,0,1))
img = np.expand_dims(img, axis=0)
                                           
out = model.predict(img)
print(np.argmax(out))

Found 10 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
1/1 [==============================] - 11s 11s/step - loss: 1.1273 - accuracy: 0.5000 - val_loss: 2.5308 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 8s 8s/step - loss: 2.4769 - accuracy: 0.5000 - val_loss: 1.5870 - val_accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 8s 8s/step - loss: 1.5717 - accuracy: 0.5000 - val_loss: 0.2603 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 8s 8s/step - loss: 0.1334 - accuracy: 1.0000 - val_loss: 0.5970 - val_accuracy: 0.6667
Epoch 5/5
1/1 [==============================] - 8s 8s/step - loss: 0.5702 - accuracy: 0.7000 - val_loss: 0.6912 - val_accuracy: 0.6667
INFO:tensorflow:Assets written to: hero_model.model/assets
0
